# Keras DIY自定义

## 自定义loss函数
一定要写成`y_true, y_pred`

In [1]:
from keras import backend as K

def MSE_loss(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=-1)

Using TensorFlow backend.


## 自定义metrics指标
与定义loss函数相似，一定要写成`y_true, y_pred`

In [2]:
from keras import backend as K

def categorical_accuracy(y_true, y_pred):
    return K.cast(K.equal(K.argmax(y_true, axis=-1),
                          K.argmax(y_pred, axis=-1)),
                  K.floatx())

## 自定义layer
以全相连网络为例。<br>
以下的是一些基础参数，必须要有，其他的可以自行添加。<br>
`bias`中的`shape`一定要写成这样`shape=(self.output_dim,)`。这是一个迭代器。

In [3]:
from keras import backend as K
from keras.engine.topology import Layer

class Linear(Layer):
    def __init__(self, output_dim,
                 **kwargs):
        super(Linear, self).__init__(**kwargs)
        self.output_dim = output_dim

    def build(self, input_shape):
        assert len(input_shape) >= 2
        input_dim = input_shape[-1]

        self.kernel = self.add_weight(
            name='weights',
            shape=(input_dim, self.output_dim),
            initializer='uniform',
            trainable=True)
        self.bias = self.add_weight(
            name='bias',
            shape=(self.output_dim,),
            initializer='uniform',
            trainable=True)

    def call(self, x):
        out = K.dot(x, self.kernel)
        out = K.bias_add(out, self.bias)
        return out

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

## 自定义 optimizer

In [4]:
from keras import backend as K
from keras.optimizers import Optimizer


class toy_SGD(Optimizer):
    def __init__(self, lr=0.01, **kwargs):
        super(toy_SGD, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')

    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]
        lr = self.lr
        for p, g in zip(params, grads):
            v = - lr * g  # velocity
            new_p = p + v
            self.updates.append(K.update(p, new_p))
        return self.updates

## Callbacks回调函数
Callbacks回调函数的作用是：对每次训练完成时（可以是一次batch，也可以是一次迭代）的进行一次自定义操作。keras的fit是默认每次batch输出训练过程。
<br>
常用回调函数有：<br>

### `keras.callbacks.ModelCheckpoint()`保存模型<br>
```
from keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint(filepath='model.hdf5', verbose=1, period=2)
```
`filepath`是目标文件路径，`verbose`是显示保存输出，`period`是保存模型迭代间隔。

### `keras.callbacks.EarlyStopping()`控制训练<br>
```
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='acc', patience=2, verbose=1, mode='auto')
```
`monitor`是监视目标，`patience`是进过多少个epoch之后，目标值不改变而停止的时间，mode可以为`auto`，`min`和`max`分别表示目标值不大幅度改变/下降/上升时停止。


### `keras.callbacks.LambdaCallback()`可以简单控制训练过程<br>
`LambdaCallback`可以在每个epoch前后(`on_epoch_begin`/`on_epoch_end`)，batch前后(`on_batch_begin`/`on_batch_end`)和训练前后(`on_train_begin`/`on_train_end`)调用其他函数。
<br>
```
from keras.callbacks import LambdaCallback

lambda_callback = LambdaCallback(
    # batch: batch_number logs: {'batch': batch_number, 'size': batch_size}/
    on_batch_begin=lambda batch, logs: print(batch, logs),

    # batch: batch_number logs: {'acc': acc_value, 'loss': loss_value, 'batch': batch_number, 'size': batch_size}
    on_batch_end=lambda batch, logs: print(batch, logs),

    # epoch_number {}
    on_epoch_begin=lambda epoch, logs: print(epoch, logs),

    # epoch_number {'acc': acc_value, 'loss': loss_value}
    on_epoch_end=lambda epoch, logs: print(epoch, logs),

    # nothing
    on_train_begin=lambda logs: print(logs),

    # {}
    on_train_end=lambda logs: print(logs),
)<br>
```

### 复写`Callback`，灵活控制训练过程<br>
```
class LossHistory(keras.callbacks.Callback):
    def __init__(self):
        pass

    def on_batch_begin(self, batch, logs=None):
        pass

    def on_batch_end(self, batch, logs=None):
        pass

    def on_epoch_begin(self, epoch, logs=None):
        pass

    def on_epoch_end(self, epoch, logs=None):
        pass

    def on_train_begin(self, logs=None):
        pass

    def on_train_end(self, logs=None):
        pass
```

In [5]:
from __future__ import print_function

# hyper-parameter
training_epoch = 100
num_classes = 10
learning_rate = 1e-3

import keras
from keras.datasets import mnist

(x_train, y_train), _ = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], -1).astype('float32') / 255
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
image_size = x_train.shape[-1]

from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(image_size,)))
model.add(Dense(10, activation='softmax'))

model.summary()

from keras.optimizers import *

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate),
              metrics=['accuracy'])


class LossHistory(keras.callbacks.Callback):
    def __init__(self):
        self.acc = {}
        self.loss = {}

    def on_epoch_end(self, epoch, logs=None):
        self.acc[epoch] = logs['acc']
        self.loss[epoch] = logs['loss']
        # model.save('model.h5')
        if (epoch + 1) % 10 == 0:
            print('Epoch %s, acc: %.5f, loss: %.5f' % (epoch+1, logs['acc'], logs['loss']))


history = LossHistory()

model.fit(x_train, y_train,
          batch_size=100,
          epochs=training_epoch,
          callbacks=[history],
          verbose=0)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
Epoch 10, acc: 0.99202, loss: 0.02823
Epoch 20, acc: 0.99868, loss: 0.00638
Epoch 30, acc: 0.99860, loss: 0.00482
Epoch 40, acc: 1.00000, loss: 0.00035
Epoch 50, acc: 1.00000, loss: 0.00014
Epoch 60, acc: 1.00000, loss: 0.00011
Epoch 70, acc: 0.99988, loss: 0.00055
Epoch 80, acc: 1.00000, loss: 0.00002
Epoch 90, acc: 1.00000, loss: 0.00002
Epoch 100, acc: 0.99993, loss: 0.00033


## Keras模型可视化
需要安装graphviz/pydot（`pip install ~`）

In [ ]:
from keras.utils.vis_utils import plot_model

plot_model(model, to_file='model.png', show_shapes=True)

## Keras还可以自定义正则化公式。再次不做展示